# Install modules

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Import statments

In [ ]:
# standard imports
import matplotlib.pyplot as plt
import os
import random
import time
import numpy as np
from PIL import Image

# transformers, hugging face
import transformers
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
from torchvision import datasets, transforms
import torchvision.transforms.functional as fn
from torch import nn
import requests
import datasets
from datasets import load_dataset

#torch
import torch
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch import nn
import torchvision

# for training
from tqdm import tqdm
import cv2
import wandb
from peft import LoraConfig
from peft import get_peft_model


# Datasets

Download FlSea Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = "/content/drive/MyDrive/MIR_Semester2/FlSeaDataset"
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"camillaleahjames","key":"d718d2279b5df283011f5dba00781d8f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pwd

/content


In [ ]:
# !kaggle datasets download -d viseaonlab/flsea-vi
!unzip -q -o /content/flsea-vi.zip -d /content/drive/MyDrive/MIR_Semester2/FlSeaDataset


[/content/flsea-vi.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/flsea-vi.zip or
        /content/flsea-vi.zip.zip, and cannot find /content/flsea-vi.zip.ZIP, period.


In [ ]:
class FlSeaDataset(Dataset):
    def __init__(self, data_dir, label_dir, transforms=None):
        self.data_dir = data_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.image_files = []
        self.label_files = []

        for root, _, files in os.walk(data_dir):
            for file in files:
                if file.endswith('.tif') or file.endswith('.tiff'):
                    image_file = os.path.join(root, file)
                    self.image_files.append(image_file)

        for root, _, files in os.walk(label_dir):
            for file in files:
                if file.endswith('.tif') or file.endswith('.tiff'):
                    label_file = os.path.join(root, file)
                    self.label_files.append(label_file)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_file = self.image_files[index]
        label_file = self.label_files[index]

        image = Image.open(image_file).convert('RGB')
        label = Image.open(label_file)

        # mask = (label != 0).astype(int)  # Create binary mask where label is not 0

        # image = image * mask

        if self.transforms:
            image = self.transforms(image)
            label  = self.transforms(label)

        return image, label


class EiffelTowerDataset(Dataset):
    def __init__(self, data_dir, label_dir, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms
        self.image_files = []
        self.mask_files = []

        for root, _, files in os.walk(data_dir):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    image_file = os.path.join(root, file)
                    self.image_files.append(image_file)

        for root, _, files in os.walk(label_dir):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    mask_file = os.path.join(root, file)
                    self.mask_files.append(mask_file)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_file = self.image_files[index]
        mask_file = self.mask_files[index]

        image = Image.open(image_file).convert('RGB')
        mask = Image.open(mask_file)

        if self.transforms:
            image = self.transforms(image)
            mask = self.transforms(mask)

        return image, mask

# Model

In [ ]:
class DepthAnythingPEFT:

    def __init__(self, model_checkpoint) -> None:
        self.model_checkpoint = model_checkpoint
        self.image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)
        self.model = AutoModelForDepthEstimation.from_pretrained(model_checkpoint,
        ignore_mismatched_sizes=True,)  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint


    def trainable_parameters(self, model):
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
        )

    def trainable_modules(self,model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name, param.shape)


    def peft_model(self, peft_config):
        return get_peft_model(self.model, peft_config)

# Training function

In [ ]:

class PEFTTraining:

    def __init__(self, model_checkpoint: str,output_dir: str, model, train_dataset: Dataset,
                 valid_dataset: Dataset, train_batch_size:int, valid_batch_size:int ,
                 loss_fn, optimizer, epoch: int, device, warmup_period_percentage,
                 learning_rate,min_lr,grad_clip, wandb_logging = True) -> None:

        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.model = model
        self.epoch = epoch
        self.output_dir = output_dir
        self.logging = wandb_logging
        self.eval = EvaluationMetric(wandb_logging)
        self.device = device
        self.model.to(self.device)
        self.train_batch_size = train_batch_size
        self.model_name = model_checkpoint.split("/")[-1]
        self.training_loader = torch.utils.data.DataLoader(train_dataset, batch_size= train_batch_size, shuffle=True)
        self.validation_loader = torch.utils.data.DataLoader(valid_dataset, batch_size= valid_batch_size, shuffle=False)
        self.num_steps = len(self.training_loader) * self.epoch
        self.learning_rate = learning_rate
        self.min_lr = min_lr
        self.grad_clip = grad_clip
        self.iter_per_epoch = len(train_dataset)/train_batch_size
        self.lr_decay_iters = int(self.iter_per_epoch * epoch) + 1
        self.warmup_period = self.lr_decay_iters*warmup_period_percentage/100
        # self.lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optimizer, T_0=T_0, T_mult=T_mult)
        # self.warmup_scheduler = warmup.LinearWarmup(self.optimizer, warmup_period=self.warmup_period)

    # learning rate decay scheduler (cosine with warmup)
    def get_lr(self,iteration):
        # 1) linear warmup for warmup_iters steps
        if iteration < self.warmup_period:
            return self.learning_rate * iteration / self.warmup_period
        # 2) if it > lr_decay_iters, return min learning rate
        if iteration > self.lr_decay_iters:
            return self.min_lr
        # 3) in between, use cosine decay down to min learning rate
        decay_ratio = (iteration - self.warmup_period) / (self.lr_decay_iters - self.warmup_period)
        assert 0 <= decay_ratio <= 1
        coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
        return self.min_lr + coeff * (self.learning_rate - self.min_lr)


    def train_one_epoch(self, epoch_index):
        running_loss = 0.
        last_loss = 0.

        # Here, we use enumerate(training_loader) instead of
        # iter(training_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        iteration = epoch_index * self.iter_per_epoch
        for i, (inputs, labels) in enumerate(self.training_loader):
            # Every data instance is an input + label pair
            inputs, labels = inputs.to(self.device),labels.to(self.device)

            # determine and set the learning rate for this iteration
            lr = self.get_lr(iteration)
            self.optimizer.param_groups[0]["lr"] = lr

            # clip the gradient
            if self.grad_clip != 0.0:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)

            # Zero your gradients for every batch!
            self.optimizer.zero_grad()

            # Make predictions for this batch
            outputs = self.model(inputs).predicted_depth

            # Compute the loss and its gradients

            loss = self.loss_fn(torch.squeeze(outputs), torch.squeeze(labels))
            loss.backward()

            # Adjust learning weights
            self.optimizer.step()

            # with self.warmup_scheduler.dampening():
            #     if self.warmup_scheduler.last_step + 1 >= self.warmup_period:
            #         self.lr_scheduler.step()

            # Gather data and report

            running_loss += loss.item()
            iteration += 1
            if i % self.train_batch_size == self.train_batch_size - 1:
                last_loss = running_loss / self.train_batch_size
                print('  batch {} loss: {}'.format(i + 1, last_loss))
                #wb_x = epoch_index * len(self.training_loader) + i + 1
                wandb.log({'Loss/train (per batch)': last_loss})
                wandb.log({'Learning Rate (per batch)':  self.optimizer.param_groups[0]["lr"]})
                running_loss = 0.

        return last_loss



    def train(self,wandb_init = None):

        epoch_number = 0

        best_vloss = 1_000_000.

        for epoch in range(self.epoch):
            print('EPOCH {}:'.format(epoch_number + 1))

            # Make sure gradient tracking is on, and do a pass over the data
            self.model.train(True)
            avg_loss = self.train_one_epoch(epoch_number)


            running_vloss = 0.0
            # Set the model to evaluation mode, disabling dropout and using population
            # statistics for batch normalization.
            self.model.eval()

            # Disable gradient computation and reduce memory consumption.
            with torch.no_grad():
                for i, (vinputs, vlabels) in enumerate(self.validation_loader):
                    vinputs, vlabels = vinputs.to(self.device),vlabels.to(self.device)
                    voutputs = self.model(vinputs).predicted_depth
                    vloss = self.loss_fn(torch.squeeze(voutputs), torch.squeeze(vlabels))
                    self.eval.compute_metrics(vinputs,voutputs,vlabels)
                    running_vloss += vloss

            avg_vloss = running_vloss / (i + 1)
            print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

            # Log the running loss averaged per batch
            # for both training and validation
            if self.logging == True:
                wandb.log({ 'Training Loss' : avg_loss, 'Validation Loss' : avg_vloss })

            # Track best performance, and save the model's state
            if avg_vloss < best_vloss:
                best_vloss = avg_vloss
                if not os.path.exists(self.output_dir):
                    os.makedirs(self.output_dir)

                model_path = '{}/{}_{}.pth'.format(self.output_dir, self.model_name, epoch_number)
                torch.save({'epoch': epoch_number,
                            'model_state_dict': self.model.state_dict(),
                            'optimizer_state_dict': self.optimizer.state_dict(),
                            'loss': best_vloss},
                            model_path)

                if self.logging == True:
                    if wandb_init is not None:
                        artifact = wandb.Artifact('model', type='model')
                        artifact.add_file(model_path)
                        wandb_init.log_artifact(artifact)
                    else:
                        print("No WandB init given; model artifact is not saved")


            epoch_number += 1



# Evaluation + Metrics

In [ ]:
wandb login
# afcce06a15ac3cc4cb2deb7200670d496bcce144 -> api key

In [ ]:
from torchvision.transforms import (
    Compose,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

### Config
EXPERIMENT_NUM = 41
MODEL_CHECKPOINT = "LiheYoung/depth-anything-small-hf"
DATASET_ROOT_DIR = "eiffel/2020/images/"
OUTPUT_DIR = f"DepthAnythingPEFT/depth-anything-small-lora_{EXPERIMENT_NUM}"
WANDB_USER = "researchpapers"
WANDB_PROJECT = "peft_training"
WANDB_DATASET = "USOD10k"

### Hyperparameters
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
DATA_USE_PERCENTAGE = 100
TRAIN_SPLIT = 0.8

LOSS = nn.L1Loss()
OPTIM = "AdamW"
EPOCH = 1
LORA_RANK = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.001
BIAS = "lora_only"
GRAD_CLIP = 1.0
MIN_LR = 1e-7

### Grid Search
LEARNING_RATE_LIST = [0.001,0.01,0.0001]
WARMUP_PERIOD_PERCENTAGE_LIST = [10,20,30,40,50]


model = DepthAnythingPEFT(model_checkpoint = MODEL_CHECKPOINT)

#consider changing the transform
data_transforms = Compose(
    [
        RandomResizedCrop(model.image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
    ]
)

dataset = EiffelTowerDataset('eiffel/2020/images/','eiffel/2020/depth/dense/depth', transforms=data_transforms)
useful_dataset_length = int(len(dataset) * DATA_USE_PERCENTAGE /100)
print(f"Length of Dataset: {useful_dataset_length}")
train_size = int(TRAIN_SPLIT * useful_dataset_length)
valid_size = useful_dataset_length - train_size
useful_dataset = Subset(dataset,list(range(useful_dataset_length)))
train_dataset, valid_dataset = torch.utils.data.random_split(useful_dataset, [train_size, valid_size])


peft_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=["query", "value"],
    lora_dropout= LORA_DROPOUT,
    bias= BIAS,
    modules_to_save=["decode_head"],
)


for LEARNING_RATE in LEARNING_RATE_LIST:

    for WARMUP_PERIOD_PERCENTAGE in WARMUP_PERIOD_PERCENTAGE_LIST:

        lora_model = model.peft_model(peft_config)
        model.trainable_parameters(lora_model)

        if OPTIM == "AdamW":
            optimizer = torch.optim.AdamW(lora_model.parameters(), lr= LEARNING_RATE)

        elif OPTIM == "SGD":
            optimizer = torch.optim.SGD(lora_model.parameters(), lr = LEARNING_RATE)

        elif OPTIM == "ADAM":
            optimizer = torch.optim.Adam(lora_model.parameters(), lr= LEARNING_RATE)

        else:
            print("Optimizer not yet implemented")


        user = WANDB_USER
        project = WANDB_PROJECT
        display_name = f"{WANDB_DATASET} lr: {LEARNING_RATE}, warmup: {WARMUP_PERIOD_PERCENTAGE}"
        config = {"lr": LEARNING_RATE, "batch_size": TRAIN_BATCH_SIZE, "data_used(%)" : DATA_USE_PERCENTAGE, "train_split": TRAIN_SPLIT, "loss": "mse",
                "optimizer" : OPTIM, "epoch": EPOCH, "lora_rank": LORA_RANK, "lora_alpha": LORA_ALPHA, "lora_dropout" :LORA_DROPOUT, "bias":BIAS,
                "warmup_period":WARMUP_PERIOD_PERCENTAGE,"min_lr": MIN_LR,"grad_clip" :GRAD_CLIP}

        logger = wandb.init(entity=user, project=project, name=display_name, config=config)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        trainer = PEFTTraining(MODEL_CHECKPOINT,OUTPUT_DIR,lora_model,train_dataset,valid_dataset,
                            TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, LOSS, optimizer, EPOCH, device,
                            WARMUP_PERIOD_PERCENTAGE,LEARNING_RATE,MIN_LR,GRAD_CLIP, True)



        trainer.train(logger)
        logger.finish()
        EXPERIMENT_NUM +=1class EvaluationMetric:

    def __init__(self, wandb_logging:bool) -> None:
        self.logging = wandb_logging


    def scale_offset(self, y_pred, y_true):
        scale_factor = np.mean(y_pred) / np.mean(y_true)

        # Adjust the second depth map by the scale factor
        true_scaled = y_pred * scale_factor

         # Calculate the offset
        offset = np.mean(y_pred) - (scale_factor * np.mean(y_true))

        # Adjust the second depth map by the offset
        true_adjusted = true_scaled + offset

        val_min = y_pred.min()
        val_range = y_pred.max() - val_min + 1e-7

        pred_normed = (y_pred - val_min) / val_range

        # apply identical normalization to the denoised image (important!)
        true_adjusted_normed = (true_adjusted - val_min) / val_range

        return pred_normed, true_adjusted_normed

    def absolute_relative_error(self, y_pred, y_true):
        """
        Calculate the Absolute Relative Error (MARE).

        Parameters:
        y_pred : torch.Tensor
            Predicted depth values.
        y_true : torch.Tensor
            Ground truth depth values.

        Returns:
        float
        Absolute Relative Error (MARE).
        """
        y_pred, y_true = self.scale_offset(y_pred, y_true)
        # mask = y_true == 0
        # y_true[mask] = 1
        absolute_relative_error = np.abs(y_pred - y_true) / y_true

        return np.mean(absolute_relative_error)

    def root_mean_squared_error(self, y_pred, y_true, log = False):

        y_pred, y_true = self.scale_offset(y_pred, y_true)
        if log:
            y_pred = np.log(y_pred)
            y_true = np.log(y_true)
        mse = np.mean((y_pred - y_true)**2)
        rmse = np.sqrt(mse)
        return rmse

    def delta1_metric(self, y_pred, y_true, threshold=1.25):
        """
        Calculate the δ1 metric for monocular depth estimation.

        Parameters:
        y_pred : torch.Tensor
            Predicted depth values.
        y_true : torch.Tensor
            Ground truth depth values.
        threshold : float, optional
            Threshold for considering a pixel as correctly estimated (default is 1.25).

        Returns:
        float
            Percentage of pixels for which max(d*/d, d/d*) < threshold.

        """
        y_pred, y_true = self.scale_offset(y_pred, y_true)
        # Compute element-wise ratios
        ratio_1 = y_true / (y_pred + 1e-7)  # Adding epsilon to avoid division by zero
        ratio_2 = (y_pred + 1e-7) / y_true  # Adding epsilon to avoid division by zero

        # Calculate element-wise maximum ratio
        max_ratio = torch.max(ratio_1, ratio_2)

        # Count the number of pixels where max_ratio < threshold
        num_correct_pixels = torch.sum(max_ratio < threshold).item()

        # Calculate the percentage of pixels satisfying the condition
        total_pixels = y_true.numel()
        percentage_correct = (num_correct_pixels / total_pixels) * 100.0

        return percentage_correct

    def si_log(y_pred, y_true):
        """
        Calculate the Scale Invarient error that takes into account the global scale of a scene.
        This metric is sensitive to the relationships between points in the scene,
        irrespective of the absolute global scale.

        Parameters:
        y_pred : torch.Tensor
            Predicted depth values.
        y_true : torch.Tensor
            Ground truth depth values.

        Returns:
        float
            SI Error

        """
        bs = y_pred.shape[0]

        y_pred = torch.reshape(y_pred, (bs, -1))
        y_true = torch.reshape(y_true, (bs, -1))

        mask = y_true > 0  # 0=missing y_true
        num_vals = mask.sum(dim=1)

        log_diff = torch.zeros_like(y_pred)
        log_diff[mask] = torch.log(y_pred[mask]) - torch.log(y_true[mask])

        si_log_unscaled = torch.sum(log_diff**2, dim=1) / num_vals - (torch.sum(log_diff, dim=1)**2) / (num_vals**2)
        si_log_score = torch.sqrt(si_log_unscaled) * 100

        si_log_score = torch.mean(si_log_score)
        return si_log_score

    def compute_metrics(self, input_image, outputs, labels):
        metrics = []

        with torch.no_grad():
            predicted_depth = outputs
            img_size = fn.get_image_size(input_image)
            img_size.reverse()
            prediction = torch.nn.functional.interpolate(
            predicted_depth.unsqueeze(1),
            size = img_size,
            mode = "bicubic",
            align_corners=False)

            # Convert depth prediction to numpy array and resize to match ground truth depth map size
            depth_output = prediction.squeeze().cpu().numpy()
            labels = labels.squeeze().cpu().numpy()

            # Handle invalid or unexpected depth values
            depth_output[depth_output <= 0] = 1e-7  # Replace negative or zero values with a small epsilon
            labels[labels <= 0] = 1e-7

            # Calculate metrics
            absRel = self.absolute_relative_error(depth_output, labels)

            rmse = self.root_mean_squared_error(depth_output, labels)
            rmseLog = self.root_mean_squared_error(depth_output, labels, log = True)

            out_t = torch.from_numpy(depth_output)
            labels_t = torch.from_numpy(labels)

            delta1 = np.mean(self.delta1_metric(out_t, labels_t))
            si_error = self.si_log(out_t, labels_t)


            if self.logging == True:
                wandb.log({"Absolute Relative error (AbsRel)": absRel})
                wandb.log({"Root Mean Squared Error (RMSE)": rmse})
                wandb.log({"Log Root Mean Squared Error (Log-RMSE)": rmseLog})
                wandb.log({"Scale Invarient Error (SI Error)": si_error})
                wandb.log({"Delta1 with thresold=1.25": delta1})

            metrics.append([absRel, rmse, rmseLog, si_error, delta1])

            return metrics


# Model configuration and experiments